## Fine-tuning 전 추론

In [ ]:
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer , pipeline
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
import re
import logging 

logging.getLogger("langchain").setLevel(logging.ERROR)


In [ ]:
def question_answer(question, model_id):
    from decouple import config
    huggingface_token = config('HUGGINGFACE_TOKEN')

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_id, huggingface_token=huggingface_token)

    hf_pipeline = pipeline(
        "text-generation", # 텍스트 생성
        model=model,
        tokenizer=tokenizer, 
        truncation=True,
        max_length=512, # 생성할 최대 토큰 길이 지정 
        temperature=0.5, # 생성의 창의성을 제어하는 매개변수
        top_p=0.9,
        repetition_penalty=1.1,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        device=0)
    
    llm = HuggingFacePipeline(pipeline=hf_pipeline)
    
    # 사용자 정의 프롬프트 템플릿 정의 
    prompt_template = """아래의 문맥을 사용하여 질문에 답하십시오. 
    만약 답을 모른다면, 모른다고 말하고 답을 지어내지 마십시오.

    질문:{question}
    답변:"""
    
    PROMPT = PromptTemplate(template=prompt_template, input_variables=['question'])
    
     # LLMChain 설정
    llm_chain = LLMChain(
        llm=llm,
        prompt=PROMPT)
    
    response = llm_chain.run(question = question)
    clean_response = response.split("답변:")[-1].strip()
    final_response = re.sub(r'</?div.*?>|</u>|</s>|</?[^>]+>|<pad>|<unk>|<mask>', '', clean_response)

    print('질문:', question)
    print('답변:', final_response)

In [ ]:
from decouple import config

huggingface_token = config('HUGGINGFACE_TOKEN')

model_id = "iljoo/iljoo-chatbot-8b-test"
question_answer('친환경 에너지를 활용하여 온실가스 배출을 줄이는 방법은 무엇인가요?',model_id)